In [86]:
# 这个脚本计算AEN，放到uv3、uv4中,转换到切线空间。
import bpy
import bmesh
import numpy as np

In [133]:
obj = bpy.context.selected_objects[0]
mesh = obj.data

In [134]:
mesh.calc_normals_split()
mesh.calc_tangents(uvmap="UVMap")
normals = []
tangents = []
bitangents = []
TBNs = []
for vert in mesh.loops:
    t = vert.tangent
    n = vert.normal
    b = vert.bitangent_sign * n.cross(t)
    TBN = np.stack((t, b, n))
    normals.append(n)
    tangents.append(t)
    bitangents.append(b)
    TBNs.append(TBN)


In [135]:
# mesh.normals_split_custom_set(bitangents)

In [136]:

def createUV(obj, i):
    if obj.data.uv_layers.get(f'UV{i}') is None:
        obj.data.uv_layers.new(name=f'UV{i}', do_init=True)


createUV(obj, 1)
createUV(obj, 2)
createUV(obj, 3)
createUV(obj, 4)



In [137]:
bpy.ops.object.mode_set(mode='EDIT')

bm = bmesh.from_edit_mesh(mesh)
bm

color = bm.loops.layers.color.get("Col")
# print(color)
if color is None:
    color = bm.loops.layers.float_color.new("Col")
uv2 = bm.loops.layers.uv["UV2"]
uv3 = bm.loops.layers.uv["UV3"]
uv4 = bm.loops.layers.uv["UV4"]


In [138]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        norm = np.finfo(v.dtype).eps
    return v / norm
# custom_normal = np.zeros((len(normals),3))
rgb=[[1,0,0,1],[0,1,0,1],[0,0,1,1],[0,0,0,1]]
for f in bm.faces:
    ci = 0
    for l in f.loops:
        # l[color] = rgb[ci]
        ci+=1
        n1 = np.zeros(3)
        n2 = np.zeros(3)
        v1, v2 = l.vert, l.link_loop_next.vert
        faces = [_l.face.index for _l in l.edge.link_loops]
        i1 = set()
        i2 = set()
        for _l in v1.link_loops:
            if _l.face.index in faces:
                i1.add(_l.index)
        for _l in v2.link_loops:
            if _l.face.index in faces:
                i2.add(_l.index)
        for i in i1:
            n1 += normals[i]
        for i in i2:
            n2 += normals[i]
        n0 = normalize(TBNs[l.index] @ normals[l.index])
        # custom_normal[l.index] = normalize(n1)
        n1 = normalize(TBNs[l.index] @ n1)
        n2 = normalize(TBNs[l.link_loop_next.index] @ n2)
        l[uv2].uv = [n0[0], n0[1]]
        l[uv3].uv = [n1[0], n1[1]]
        l.link_loop_next[uv4].uv = [n2[0], n2[1]]
        # print(n1)

bmesh.update_edit_mesh(mesh)
bpy.ops.object.mode_set(mode='OBJECT')



{'FINISHED'}

In [102]:
# mesh.normals_split_custom_set(custom_normal)